In [1]:
# in this approach we will use short-based boxes to draw line to the ankle points

In [23]:
import pandas as pd
import numpy as np
from matplotlib.pyplot import imshow
from tqdm import tqdm

In [3]:
df = pd.read_pickle('df.pkl')

In [4]:
df.head(1)

,human_parsing,image,pose_img,pose_json,pose_keypoints,convex_skirts,mask_shirts_closed,mask_upper_body,mask_legs_and_shorts,mask_shirts
0,<PIL.Image.Image image mode=RGB size=768x1024 ...,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,version ...,"[422.798, 7.55812, 0.125119, 394.295, 79.8684,...","[[False, False, False, False, False, False, Fa...","[[False, False, False, False, False, False, Fa...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [18]:
short_colors = {(64, 0, 128), (192, 0, 0)}

In [19]:
def get_color_mask(img, colors):
    img = np.array(img)
    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    for color in colors:
        mask += np.all(img == color, axis=-1).astype(np.uint8)
    return mask

In [24]:
df['mask_shorts'] = [get_color_mask(a, short_colors) for a in tqdm(df['human_parsing'])]

100%|██████████| 704/704 [00:28<00:00, 25.00it/s]


In [25]:
# make 4 point arrays from mask_shorts
def get_box_points(mask):
    mask = np.array(mask)
    points = np.argwhere(mask)
    x_min, y_min = points.min(axis=0)
    x_max, y_max = points.max(axis=0)
    return np.array([x_min, y_min, x_max, y_max])

In [26]:
df['shorts_boxes'] = [get_box_points(a) for a in tqdm(df['mask_shorts'])]

100%|██████████| 704/704 [00:03<00:00, 192.33it/s]


In [46]:
df.pose_json[0].people[0]['pose_keypoints_2d'] # 3 for each point (x, y, confidence)

[422.798,
 7.55812,
 0.125119,
 394.295,
 79.8684,
 0.8518159999999999,
 303.66,
 79.7733,
 0.7621100000000001,
 320.732,
 215.98,
 0.836236,
 487.858,
 193.199,
 0.8460759999999999,
 476.559,
 79.9503,
 0.799931,
 541.739,
 218.774,
 0.7829969999999999,
 371.733,
 221.548,
 0.501814,
 414.184,
 340.53,
 0.553848,
 351.829,
 340.551,
 0.5503800000000001,
 309.427,
 598.319,
 0.7281949999999999,
 235.8,
 896.071,
 0.7591249999999999,
 465.346,
 337.705,
 0.517478,
 462.247,
 626.72,
 0.7748919999999999,
 411.386,
 864.817,
 0.7735639999999999,
 0,
 0,
 0,
 0,
 0,
 0,
 349.047,
 7.57927,
 0.0774477,
 0,
 0,
 0,
 436.87,
 941.286,
 0.6640969999999999,
 453.918,
 932.792,
 0.666011,
 385.891,
 870.418,
 0.743521,
 230.008,
 969.691,
 0.6601549999999999,
 210.156,
 963.988,
 0.647822,
 247.063,
 910.171,
 0.414717]

In [40]:
get_pose(df['pose_json'][0])[11]

array([2.35800e+02, 8.96071e+02, 7.59125e-01])